In [2]:
import numpy as np
import pandas as pd

#data = pd.read_csv("rental_data_cleaned.csv")
data = pd.read_csv("all___rental_data_cleaned.csv")

data = data[data['Rent']>100]
data = data[data['Rent']<10000]
data = data[data['house_size_sqft']<6000]

data["bed_rooms"] = data["bed_rooms"].fillna(data["bed_rooms"].mean())
data["house_size_sqft"] = data["house_size_sqft"].fillna(data["house_size_sqft"].mean())

data["bed_rooms"] = data["bed_rooms"].astype(int)
data["house_size_sqft"] = data["house_size_sqft"].astype(int)

len(data)

10669

In [3]:
req_data = data[["Post_ID","Area","bed_rooms","house_size_sqft","Rent"]]

recom_data = req_data.drop('Post_ID', axis=1)

recom_data = pd.get_dummies(recom_data)

recom_data.head(10)

,bed_rooms,house_size_sqft,Rent,Area_V1E,Area_V1K,Area_V1M,Area_V1T,Area_V1V,Area_V1W,Area_V1X,...,Area_V9L,Area_V9M,Area_V9N,Area_V9P,Area_V9R,Area_V9S,Area_V9T,Area_V9V,Area_V9Y,Area_V9Z
0,1,540,850.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,2000,1950.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,900,1450.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,2,900,1295.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,2,900,925.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,1,1000,1050.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,2,1040,1500.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,3,1500,2400.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24,2,1034,2150.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,2,1111,2400.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
area_list = list(data['Area'].unique())

In [5]:
#Input Data Creation

def get_p_area(samp_area):
    k = data[data["Area_Name"]==samp_area]
    return k.Area


samp_bed = 3 

samp_area_sqft = 1500

# use a drop down - for area - not returning unique values - using postal area directly
samp_area = "Port Coquitlam"

samp_postal_area = "V3L" #get_p_area(samp_area) # have to change this 

sample_df = pd.DataFrame({"bed_rooms":samp_bed,\
                          "house_size_sqft":samp_area_sqft},index=[0])

sample_df["Area_"+samp_postal_area]=1

area_cols = set(area_list)-(set({samp_postal_area}))

for col in area_cols:
    sample_df["Area_"+col]=0

#Input DF    
sample_df

,bed_rooms,house_size_sqft,Area_V3L,Area_V9R,Area_V3G,Area_V5N,Area_V9C,Area_V6M,Area_V3Z,Area_V8Z,...,Area_V2S,Area_V5A,Area_V1E,Area_V6L,Area_V6W,Area_V6Y,Area_V7L,Area_V7X,Area_V4P,Area_V4T
0,3,1500,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
sample_df_with_price = sample_df.copy()
sample_df_with_price['Rent'] = 2000

In [7]:
recom_data_no_rent = recom_data.drop('Rent', axis=1)

#Add this row on top of all records
#recom_numeric = pd.concat([sample_df, recom_data_no_rent], axis=0,sort=False)

#With Price
recom_numeric = pd.concat([sample_df_with_price, recom_data], axis=0,sort=False)

#Normalize
recom_normal = (recom_numeric - recom_numeric.min()) / (recom_numeric.max() - recom_numeric.min())

#Normalized sample
normal_sample = recom_normal.iloc[0,:]

#normalized sim_data
recom_normal = recom_normal.iloc[1:,:]

In [8]:
from sklearn.neighbors import NearestNeighbors

#train model
nbrs = NearestNeighbors(n_neighbors=10, metric='euclidean',algorithm='auto').fit(recom_normal)

In [9]:
import pickle
# now you can save it to a file
with open('rental_knn_model.pkl', 'wb') as f:
    pickle.dump(nbrs, f)

# and later you can load it
with open('rental_knn_model.pkl', 'rb') as f:
    loaded_model_knn = pickle.load(f)

In [10]:
# get model results on smaple data
vals = loaded_model_knn.kneighbors([normal_sample])

# get indices
indices = list(vals[1][0])
indices[:3]
print(len(indices))

10


In [11]:
sample_df

,bed_rooms,house_size_sqft,Area_V3L,Area_V9R,Area_V3G,Area_V5N,Area_V9C,Area_V6M,Area_V3Z,Area_V8Z,...,Area_V2S,Area_V5A,Area_V1E,Area_V6L,Area_V6W,Area_V6Y,Area_V7L,Area_V7X,Area_V4P,Area_V4T
0,3,1500,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# KNN - 5 Similar records
recom_data.iloc[indices,:]

,bed_rooms,house_size_sqft,Rent,Area_V1E,Area_V1K,Area_V1M,Area_V1T,Area_V1V,Area_V1W,Area_V1X,...,Area_V9L,Area_V9M,Area_V9N,Area_V9P,Area_V9R,Area_V9S,Area_V9T,Area_V9V,Area_V9Y,Area_V9Z
10571,3,1400,1850.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2150,3,1700,2100.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2146,3,1300,2200.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2143,3,1579,2500.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2147,3,1300,1600.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10534,3,1100,2100.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10553,3,1100,1895.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2148,3,1300,1295.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10583,3,1100,1580.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2128,3,1250,2700.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Required Records
req_records = data.iloc[indices,:]
req_records

,Post_ID,Time,Area,Area_Name,Address,bed_rooms,house_size_sqft,Rent
10571,6846407302,2019-03-20 16:38:00,V3L,New Westminster,7949-10th Ave Burnaby,3,1400,1850.0
2150,6823663826,2019-02-23 00:41:00,V3L,New Westminster,NEW WESTMINSTER,3,1700,2100.0
2146,6820116403,2019-02-23 08:28:00,V3L,New Westminster,Burnaby,3,1300,2200.0
2143,6819774765,2019-02-23 17:19:00,V3L,New Westminster,Queen's Park in New Westminster,3,1579,2500.0
2147,6824423944,2019-02-23 08:28:00,V3L,New Westminster,Burnaby,3,1300,1600.0
10534,6852568453,2019-03-28 10:59:00,V3L,New Westminster,New Westminster,3,1100,2100.0
10553,6850229889,2019-03-25 13:03:00,V3L,New Westminster,new westminster next to RCH and skytrain,3,1100,1895.0
2148,6826252829,2019-02-23 07:29:00,V3L,New Westminster,short walk from Skytrain,3,1300,1295.0
10583,6828918581,2019-03-10 14:26:00,V3L,New Westminster,Burnaby,3,1100,1580.0
2128,6821045445,2019-02-24 11:56:00,V3L,New Westminster,New Westminster,3,1250,2700.0


In [14]:
# For 3 bed-1500 sqft-Penticton Area - 1450
# For 5 bed-3000 sqft-New WestMinister Area - 2800.0
# For 3 bed-1500 sqft-New WestMinister Area -2350
Estimated_price = req_records["Rent"].median()
Estimated_price

1997.5

## Try a different Approach

In [15]:
from sklearn.metrics import mean_squared_error
from math import sqrt # rms = sqrt(mean_squared_error(y_true, y_predicted))
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd

# linear regression models

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet,BayesianRidge
from sklearn.svm import SVR

# cross val, k-folds
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error
from sklearn.pipeline import make_pipeline

In [16]:
import seaborn as sns

analyze_data = data.drop('Post_ID', axis=1)
corr = analyze_data.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True)

In [17]:
corr

,bed_rooms,house_size_sqft,Rent
bed_rooms,1.000000,0.80514,0.494042
house_size_sqft,0.805140,1.00000,0.603520
Rent,0.494042,0.60352,1.000000


In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
recom_data[['bed_rooms', 'house_size_sqft']] = scaler.fit_transform(recom_data[['bed_rooms', 'house_size_sqft']])

recom_data.head()

c:\users\rohithsoorampc\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,bed_rooms,house_size_sqft,Rent,Area_V1E,Area_V1K,Area_V1M,Area_V1T,Area_V1V,Area_V1W,Area_V1X,...,Area_V9L,Area_V9M,Area_V9N,Area_V9P,Area_V9R,Area_V9S,Area_V9T,Area_V9V,Area_V9Y,Area_V9Z
0,0.000000,0.090209,850.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.428571,0.334561,1950.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0.285714,0.150460,1450.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0.142857,0.150460,1295.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0.142857,0.150460,925.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
y = recom_data['Rent']
X = recom_data.loc[:, recom_data.columns != 'Rent']
X = pd.get_dummies(X)

In [20]:
X_copy,y_copy = X.copy(),y.copy()

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_copy, y_copy, test_size=0.2,random_state=23)

In [21]:
grid_param = {'normalize':[True,False]}

'''grid_param = {  
    'n_estimators': [100, 300, 500, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False]
}'''

lr = LinearRegression()

grid_search = GridSearchCV(estimator= lr,  
                     param_grid=grid_param,
                     scoring='neg_mean_squared_error',
                     cv=5,
                     n_jobs=-1)

grid_search.fit(X,y)

best_parameters = grid_search.best_params_  
print("Linear Regression Best Parameters:",best_parameters)

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

In [161]:
grid_param = {
    'n_iter':[100,300,500],
    'alpha_1':[1.e-6,1.e-7],
    'alpha_2':[1.e-6,1.e-7],
    'lambda_1':[1.e-6,1.e-7],
    'lambda_2':[1.e-6,1.e-7],
}

br = BayesianRidge()

grid_search = GridSearchCV(estimator= br,  
                     param_grid=grid_param,
                     scoring='neg_mean_squared_error',
                     cv=5,
                     n_jobs=-1)

grid_search.fit(X,y)

best_parameters = grid_search.best_params_  
print("Bayesian Ridge Best Parameters:",best_parameters)

Bayesian Ridge Best Parameters: {'alpha_1': 1e-07, 'alpha_2': 1e-06, 'lambda_1': 1e-06, 'lambda_2': 1e-07, 'n_iter': 100}


In [162]:
grid_param = {
    'max_depth':[None,2,4],
    'min_samples_split':[2,4,6],
    'min_samples_leaf':[1,2,3],
    'max_features':["auto","sqrt","log2"]
}

dtr = DecisionTreeRegressor()

grid_search = GridSearchCV(estimator= dtr,  
                     param_grid=grid_param,
                     scoring='neg_mean_squared_error',
                     cv=5,
                     n_jobs=-1)

grid_search.fit(X,y)

best_parameters = grid_search.best_params_  
print("Decision Tree Best Parameters:",best_parameters)

Decision Tree Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 6}


In [163]:
grid_param = {
    'n_estimators':[10,100,500],
    'max_depth':[2],
    'min_samples_split':[6],
    'min_samples_leaf':[2],
    'max_features':["auto","sqrt","log2"]
}

rfr = RandomForestRegressor()

grid_search = GridSearchCV(estimator= rfr,  
                     param_grid=grid_param,
                     scoring='neg_mean_squared_error',
                     cv=5,
                     n_jobs=-1)

grid_search.fit(X,y)

best_parameters = grid_search.best_params_ 

print("Random Forest Best Parameters:",best_parameters)

Random Forest Best Parameters: {'max_depth': 2, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 100}


In [165]:
grid_param = {
    'C':[0.1,1],
    'max_iter':[500,1000],
    'kernel':['rbf','linear','poly'],
    'gamma':[0.01,0.1,0.2],
    'degree':[3,4],
}

svr = SVR()

grid_search = GridSearchCV(estimator= svr,  
                     param_grid=grid_param,
                     scoring='neg_mean_squared_error',
                     cv=5,
                     n_jobs=-1)

grid_search.fit(X,y)

best_parameters = grid_search.best_params_ 

print("SVR Best Parameters:",best_parameters)

exception calling callback for <Future at 0x1680fc246d8 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "c:\users\rohithsoorampc\appdata\local\programs\python\python36\lib\site-packages\sklearn\externals\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "c:\users\rohithsoorampc\appdata\local\programs\python\python36\lib\site-packages\sklearn\externals\joblib\parallel.py", line 309, in __call__
    self.parallel.dispatch_next()
  File "c:\users\rohithsoorampc\appdata\local\programs\python\python36\lib\site-packages\sklearn\externals\joblib\parallel.py", line 731, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "c:\users\rohithsoorampc\appdata\local\programs\python\python36\lib\site-packages\sklearn\externals\joblib\parallel.py", line 759, in dispatch_one_batch
    self._dispatch(tasks)
  File "c:\users\rohithsoorampc\appdata\local\programs\python\python36\lib\site-packages

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

In [164]:
grid_param = {
    'n_estimators':[30,50,100],
    'learning_rate':[0.5,1,2],
    'loss':['linear','square']
}

abr = AdaBoostRegressor()

grid_search = GridSearchCV(estimator= abr,  
                     param_grid=grid_param,
                     scoring='neg_mean_squared_error',
                     cv=5,
                     n_jobs=-1)

grid_search.fit(X,y)

best_parameters = grid_search.best_params_ 

print("Ada Boost Best Parameters:",best_parameters)

Ada Boost Best Parameters: {'learning_rate': 0.5, 'loss': 'linear', 'n_estimators': 30}


In [166]:
grid_param = {
    'n_estimators':[100,300],
    'learning_rate':[0.05,0.1,0.2],
    'loss':['ls','huber'],
    'max_features':["auto","sqrt","log2"],
    'max_depth':[2,3]
}

gbr = GradientBoostingRegressor()

grid_search = GridSearchCV(estimator= gbr,  
                     param_grid=grid_param,
                     scoring='neg_mean_squared_error',
                     cv=5,
                     n_jobs=-1)

grid_search.fit(X,y)

best_parameters = grid_search.best_params_ 

print("Gradient Boost Best Parameters:",best_parameters)

Gradient Boost Best Parameters: {'learning_rate': 0.2, 'loss': 'huber', 'max_depth': 3, 'max_features': 'auto', 'n_estimators': 300}


In [22]:
lm_model = LinearRegression()
br_model = BayesianRidge(alpha_1=1e-07, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-07, n_iter=100)
dtr_model = DecisionTreeRegressor(max_depth=None, max_features='sqrt', min_samples_leaf=3, min_samples_split=6)
rfr_model = RandomForestRegressor(max_depth=2, max_features='auto', min_samples_leaf=2, min_samples_split=6,n_estimators=100)
svr_model = SVR(C=1, degree=4, gamma=0.2, kernel='poly',max_iter=1000)
abr_model = AdaBoostRegressor(learning_rate=0.5, loss='linear',n_estimators=30)
gbr_model = GradientBoostingRegressor(learning_rate=0.2, loss='huber', max_depth=3, max_features='auto',n_estimators=300 , random_state=123)
gbr_model_2 = GradientBoostingRegressor()

In [23]:
lm_model.fit(X_train,y_train)
br_model.fit(X_train,y_train)
dtr_model.fit(X_train,y_train)
rfr_model.fit(X_train,y_train)
svr_model.fit(X_train,y_train)
abr_model.fit(X_train,y_train)
gbr_model.fit(X_train,y_train)
gbr_model_2.fit(X_train,y_train)

c:\users\rohithsoorampc\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [24]:
y_pred = lm_model.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred)) 
r2 = r2_score(y_test, y_pred)

print("Linear Regression Scores- rmse:",rmse," r2:",r2)

y_pred = br_model.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred)) 
r2 = r2_score(y_test, y_pred)

print("Bayesian Ridge Scores- rmse:",rmse," r2:",r2)

y_pred = dtr_model.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred)) 
r2 = r2_score(y_test, y_pred)

print("Decision Tree Scores- rmse:",rmse," r2:",r2)

y_pred = rfr_model.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred)) 
r2 = r2_score(y_test, y_pred)

print("Random Forest Scores- rmse:",rmse," r2:",r2)

y_pred = svr_model.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred)) 
r2 = r2_score(y_test, y_pred)

print("Support Vector Regression Scores- rmse:",rmse," r2:",r2)

y_pred = abr_model.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred)) 
r2 = r2_score(y_test, y_pred)

print("Ada Boost Regression Scores- rmse:",rmse," r2:",r2)

y_pred_1 = gbr_model.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred_1)) 
r2 = r2_score(y_test, y_pred_1)

print("Gradient Boost Regression Scores- rmse:",rmse," r2:",r2)

y_pred = gbr_model_2.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred)) 
r2 = r2_score(y_test, y_pred)

print("Gradient Boost(No tuning) Regression Scores- rmse:",rmse," r2:",r2)

Linear Regression Scores- rmse: 832.2707286575835  r2: 0.5711821631588081
Bayesian Ridge Scores- rmse: 831.0923498994551  r2: 0.5723955954920339
Decision Tree Scores- rmse: 864.0158922014028  r2: 0.5378456320892726
Random Forest Scores- rmse: 1000.0525431229127  r2: 0.3808594655145001
Support Vector Regression Scores- rmse: 1292.3888132501488  r2: -0.03402248079355563
Ada Boost Regression Scores- rmse: 1021.4224967128291  r2: 0.3541161306213769
Gradient Boost Regression Scores- rmse: 759.9966041032839  r2: 0.6424252003112294
Gradient Boost(No tuning) Regression Scores- rmse: 803.7097539342298  r2: 0.6001085868849598


In [25]:
import pickle
# now you can save it to a file
with open('gbr_model_rental.pkl', 'wb') as f:
    pickle.dump(gbr_model, f)

# and later you can load it
with open('gbr_model_rental.pkl', 'rb') as f:
    loaded_model_GBT = pickle.load(f)

In [26]:
#Neural Network
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

from keras.regularizers import l1
from keras.layers.normalization import BatchNormalization

model_Neural = Sequential()

# get # of columns in training data
cols = X_train.shape[1]

#adding layers
model_Neural.add(Dense(58, activation='relu', input_shape=(cols,)))
model_Neural.add(Dropout(0.2))
model_Neural.add(Dense(30, activation='relu'))
model_Neural.add(Dropout(0.2))
model_Neural.add(Dense(10, activation='relu'))
model_Neural.add(Dense(1))

model_Neural.compile(optimizer='adam', loss='mse')

early_stopping_monitor = EarlyStopping(patience=5)

model_Neural.fit(X_train, y_train, validation_split=0.2, epochs=200, callbacks=[early_stopping_monitor])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 6828 samples, validate on 1707 samples
Epoch 1/200
6828/6828 [==============================] - 0s 66us/step - loss: 7126689.0129 - val_loss: 5624970.5539
Epoch 2/200
6828/6828 [==============================] - 0s 33us/step - loss: 2604695.7508 - val_loss: 1002811.2190
Epoch 3/200
6828/6828 [==============================] - 0s 33us/step - loss: 1082570.0949 - val_loss: 888376.3136
Epoch 4/200
6828/6828 [==============================] - 0s 34us/step - loss: 985351.3730 - val_loss: 824126.2665
Epoch 5/200
6828/6828 [==============================] - 0s 33us/step - loss: 913837.7488 - val_loss: 784242.3154
Epoch 6/200
6828/6828 [==============================] - 0s 33us/step - loss: 875716.7016 - val_loss: 759322.3785
Epoch 7/200
6828/6828 [=====

6828/6828 [==============================] - 0s 30us/step - loss: 721587.1069 - val_loss: 642009.7716
Epoch 64/200
6828/6828 [==============================] - 0s 30us/step - loss: 709513.1094 - val_loss: 653456.1973
Epoch 65/200
6828/6828 [==============================] - 0s 31us/step - loss: 705361.1585 - val_loss: 641881.5331
Epoch 66/200
6828/6828 [==============================] - 0s 33us/step - loss: 719829.9109 - val_loss: 649644.3755
Epoch 67/200
6828/6828 [==============================] - 0s 33us/step - loss: 718288.4052 - val_loss: 650251.9413
Epoch 68/200
6828/6828 [==============================] - 0s 33us/step - loss: 706453.0685 - val_loss: 641964.2571
Epoch 69/200
6828/6828 [==============================] - 0s 33us/step - loss: 709056.4649 - val_loss: 640060.3797
Epoch 70/200
6828/6828 [==============================] - 0s 33us/step - loss: 714477.0283 - val_loss: 640198.5274
Epoch 71/200
6828/6828 [==============================] - 0s 32us/step - loss: 710337.5284 - 

In [27]:
predictions = model_Neural.predict(X_test)
print("R2 score:",r2_score(y_test, predictions))
rms = sqrt(mean_squared_error(y_test, predictions))
print("RMSE Score:",rms)

R2 score: 0.601905708342791
RMSE Score: 801.9017750612273


In [28]:
avg_Ypred = []
for i,j in zip(y_pred,predictions):
    Average_pred =(i+j)/2
    avg_Ypred.append(int(Average_pred))

#Assigning for NDCG
test_Y_pred=avg_Ypred

In [29]:
#On average
print("R2 score:",r2_score(y_test, test_Y_pred))
rms_avg = sqrt(mean_squared_error(y_test, test_Y_pred))
print("RMSE Score:",rms_avg)

R2 score: 0.6211100678186009
RMSE Score: 782.3205391402154


## Stacking

In [33]:
col1 = rfr_model.predict(X)
col2 = br_model.predict(X)
col3 = gbr_model.predict(X)
col4 = model_Neural.predict(X)
col4 = [i[0] for i in col4]



df = pd.DataFrame(columns=['col1','col2','col3','col4'])

for a,b,c,d in zip(col1,col2,col3,col4):
    df = df.append({'col1': a, 'col2': b, 'col3': c, 'col4':d },ignore_index=True)

df.head()

,col1,col2,col3,col4
0,1954.815602,711.121238,991.974183,813.323975
1,3300.320977,2470.921812,2201.976940,2174.121826
2,1971.515577,1350.377662,1404.193296,1276.612427
3,1971.515577,1187.450394,1306.620444,1171.791992
4,1971.515577,1187.450394,1306.620444,1171.791992


In [34]:
len(y)

10669

In [37]:
X_new = df.copy()

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2,random_state=23)

model = GradientBoostingRegressor()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred)) 
r2 = r2_score(y_test, y_pred)

print("regression Scores- rmse:",rmse," r2:",r2)

regression Scores- rmse: 770.3794049540519  r2: 0.6325883452267482


## KNN Regressor

In [ ]:
#Try

## Plot 3d

In [310]:
data = pd.read_csv("all___rental_data_cleaned.csv")

data = data[data['Rent']>100]
data = data[data['Rent']<10000]
data = data[data['house_size_sqft']<6000]

data["bed_rooms"] = data["bed_rooms"].fillna(data["bed_rooms"].mean())
data["house_size_sqft"] = data["house_size_sqft"].fillna(data["house_size_sqft"].mean())

data["bed_rooms"] = data["bed_rooms"].astype(int)
data["house_size_sqft"] = data["house_size_sqft"].astype(int)

In [311]:
req_data = data[["Post_ID","Area","bed_rooms","house_size_sqft","Rent"]]

recom_data = req_data.drop('Post_ID', axis=1)

recom_data.head()

,Area,bed_rooms,house_size_sqft,Rent
0,V2A,1,540,850.0
3,V2A,4,2000,1950.0
7,V2A,3,900,1450.0
15,V2A,2,900,1295.0
19,V2A,2,900,925.0


In [312]:
sample_df

,bed_rooms,house_size_sqft,Area_V3L,Area_V1T,Area_V6G,Area_V4B,Area_V5T,Area_V5G,Area_V2E,Area_V9V,...,Area_V4C,Area_V6H,Area_V2R,Area_V6N,Area_V9Y,Area_V6K,Area_V7W,Area_V5K,Area_V9T,Area_V6B
0,3,1500,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [313]:
req_records

,Post_ID,Time,Area,Area_Name,Address,bed_rooms,house_size_sqft,Rent
10571,6846407302,2019-03-20 16:38:00,V3L,New Westminster,7949-10th Ave Burnaby,3,1400,1850.0
2150,6823663826,2019-02-23 00:41:00,V3L,New Westminster,NEW WESTMINSTER,3,1700,2100.0
2146,6820116403,2019-02-23 08:28:00,V3L,New Westminster,Burnaby,3,1300,2200.0
2143,6819774765,2019-02-23 17:19:00,V3L,New Westminster,Queen's Park in New Westminster,3,1579,2500.0
2147,6824423944,2019-02-23 08:28:00,V3L,New Westminster,Burnaby,3,1300,1600.0
10534,6852568453,2019-03-28 10:59:00,V3L,New Westminster,New Westminster,3,1100,2100.0
10553,6850229889,2019-03-25 13:03:00,V3L,New Westminster,new westminster next to RCH and skytrain,3,1100,1895.0
2148,6826252829,2019-02-23 07:29:00,V3L,New Westminster,short walk from Skytrain,3,1300,1295.0
10583,6828918581,2019-03-10 14:26:00,V3L,New Westminster,Burnaby,3,1100,1580.0
2128,6821045445,2019-02-24 11:56:00,V3L,New Westminster,New Westminster,3,1250,2700.0


In [315]:
import plotly.offline as pyo
import plotly.graph_objs as go

import numpy as np

# For 3 bed-1500 sqft-New WestMinister Area -2350

#area_V3L = recom_data[recom_data.Area =='V3L']
area_V3L = data[data.Area_Name =='New Westminster']

x, y, z = area_V3L['bed_rooms'],area_V3L['house_size_sqft'],area_V3L['Rent']

trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    name='All Data points',
    marker=dict(
        size=8,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.3
        ),
        opacity=0.4
    )
)

x2, y2, z2 = req_records['bed_rooms'],req_records['house_size_sqft'],req_records['Rent']
trace2 = go.Scatter3d(
    x=x2,
    y=y2,
    z=z2,
    mode='markers',
    name='Nearest Neighbours',
    marker=dict(
        color='green',
        size=10,
        symbol='circle',
        line=dict(
            color='green',
            width=1
        ),
        opacity=1
    )
)

x3,y3,z3= [3],[1500],[2000]
trace3 = go.Scatter3d(
    x=x3,
    y=y3,
    z=z3,
    mode='markers',
    name='Sample Data',
    marker=dict(
        color='red',
        size=10,
        symbol='circle',
        line=dict(
            color='red',
            width=1
        ),
        opacity=1
    )
)

plotly_data = [trace1,trace2,trace3]
layout = go.Layout(
    title='Plot----Title',
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    scene = dict(
    xaxis = dict(
        title='Bed Rooms'),
    yaxis = dict(
        title='Area(sqft)'),
    zaxis = dict(
        title='Rent'),),
    legend=dict(x=0.7, y=0.7)
)
fig = go.Figure(data=plotly_data, layout=layout)
pyo.plot(fig, filename='simple-3d-scatter')

'file://C:\\Users\\RohithSooramPC\\Downloads\\BigA\\CMPT-733-Project - My Changes\\data\\simple-3d-scatter.html'